In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

## Store CSV into DataFrame, Extract & Transform
**We have five csv file:**
* Airlines - has information about airline_id(IATA Code) and airline name;
* Airports - has information about airport_id, airport, city, state, country, latitude, longitude
* Flights  - contains information about date, day of the week,airline id, flight number,tail number,origin airport, destination airport, scheduled departure, actual departure time,delay(in minutes), scheduled arrival, Arrival delay, and cancellation.
* tweets   - information about tweets regarding the flight experience,ariline and date of tweets.
* there is one additional file: cancellation reason. 

# 1. Airline

In [ ]:
airline_file = "./Resources/airlines.csv"
airline_df = pd.read_csv(airline_file)
airline_df

In [ ]:
airline_df.columns = ['AIRLINE_ID','AIRLINE']
# airline_df.set_index('AIRLINE_ID',inplace=True)
airline_df

# 2. Airport

In [ ]:
airports_file = "./Resources/airports.csv"
airports_df = pd.read_csv(airports_file)
airports_df.columns =['AIRPORT_ID','AIRPORT','CITY','STATE','COUNTRY','LATITUDE','LONGITUDE']
# airports_df.set_index('AIRPORT_ID',inplace=True)
airports_df.head()

# 3. Flight

In [58]:
flights_file = "./Resources/flights.csv"
flights_df = pd.read_csv(flights_file)
flights_df.head(5)

C:\Users\DELGCA01\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Check Datatypes of all the columns

In [ ]:
flights_df.dtypes

## Subset the airlines matching the tweets data

In [ ]:
flights_df = flights_df.loc[(flights_df['AIRLINE']=='UA')| (flights_df['AIRLINE']=='US')| (flights_df['AIRLINE']=='AA')
                            | (flights_df['AIRLINE']=='WN') | (flights_df['AIRLINE']=='VX')]

### Check NaN values

In [ ]:
# Drop rows of NaN
# There isn't a row that's completely NaN
all_nan_df = flights_df[flights_df.isna().all(axis=1)]
all_nan_df.head()

In [ ]:
# However, for the flights that's not delayed/ cancelled, those columns are NaN
any_nan_df = flights_df[flights_df.isna().any(axis=1)]
any_nan_df.head()

In [ ]:
# For readibility, we replace NaN with 0 
flights_df = flights_df.fillna(0)
flights_df[flights_df.isna().any(axis=1)]

# For excluding 1 column: any_nan_df.loc[:, any_nan_df.columns != 'CANCELLATION_REASON'].fillna(0)

### Modify Columns and Labels of categories

In [ ]:
# DROP DELAY columns
flights_df = flights_df.drop(columns=['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY',
                         'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
flights_df

In [ ]:
# Merge DateTime to one column
flights_df['DATE'] = pd.to_datetime(flights_df[['YEAR','MONTH','DAY']])
flights_df = flights_df.drop(columns=['YEAR','MONTH','DAY'])
flights_df.head()

In [ ]:
# Saturday has the least flights
# Thursday has the most flights
flights_df.groupby('DAY_OF_WEEK').count()

In [ ]:
# Change Day of week from num to string
flights_df.loc[flights_df["DAY_OF_WEEK"] == 1, "DAY_OF_WEEK"] = "Monday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 2, "DAY_OF_WEEK"] = "Tuesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 3, "DAY_OF_WEEK"] = "Wednesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 4, "DAY_OF_WEEK"] = "Thursday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 5, "DAY_OF_WEEK"] = "Friday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 6, "DAY_OF_WEEK"] = "Saturday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 7, "DAY_OF_WEEK"] = "Sunday"

### Merge with airline and airport df

In [ ]:
flights_air = pd.merge(flights_df,airline_df,left_on='AIRLINE',right_on='AIRLINE_ID',how='left')
flights_air = flights_air.drop(columns=['AIRLINE_ID'])
flights_air = flights_air.rename(columns={'AIRLINE_x': 'AIRLINE_ID', 'AIRLINE_y': 'AIRLINE'})
flights_air

### Delayed Flights

In [ ]:
# The total df of delay flights
arrival_Delay = flights_air.loc[flights_air['ARRIVAL_DELAY'] > 0].reset_index(drop=True)  # negative value means arrive earlier
departure_Delay=flights_air.loc[flights_air['DEPARTURE_DELAY']> 0].reset_index(drop=True)

#  for dep_Delay: 2125618 rows
#  for arr_Delay: 2086896 rows

In [ ]:
# Both arrival and departure are delayed
delay_Flights = flights_air.loc[(flights_air['ARRIVAL_DELAY'] > 0) & (flights_air['DEPARTURE_DELAY'] > 0)].reset_index(drop=True)
delay_Flights  
# 1508147 total

In [ ]:
# Further Narrow down the dataframe
delay_Flights = delay_Flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','TAIL_NUMBER','FLIGHT_NUMBER','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT','DEPARTURE_DELAY','ARRIVAL_DELAY']]
delay_Flights.head(5)
# We are going to merge this dataset with tweets below.

### Cancelled Flights

In [ ]:
# Find out the values within the column
flights_air['CANCELLED'].value_counts()

In [ ]:
cancelled_flights = flights_air.loc[flights_air['CANCELLED']==1].reset_index(drop=True)

In [ ]:
# Count total number of cancelled flights
cancelled_flights.count()['CANCELLED']

In [ ]:
# Remove irrelevant columns 
cancelled_flights = cancelled_flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','TAIL_NUMBER','FLIGHT_NUMBER','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT','DEPARTURE_DELAY','ARRIVAL_DELAY']]
cancelled_flights.head(5)

# 4. Tweets data

In [ ]:
tweets_file = "./Resources/Tweets.csv"
tweets_df = pd.read_csv(tweets_file)
tweets_df.head(5)

In [ ]:
tweets_df = tweets_df[['tweet_id','airline_sentiment','negativereason','airline','tweet_created']]
tweets_df.head(5)

In [ ]:
tweets_df['airline'].value_counts()

In [ ]:
# Update the airline Name so it matches the flights_df
tweets_df.loc[tweets_df["airline"] == "American", "AIRLINE"] = "American Airlines Inc."
tweets_df.loc[tweets_df["airline"] == "Delta",    "AIRLINE"] = "Delta Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Southwest","AIRLINE"] = "Southwest Airlines Co."
tweets_df.loc[tweets_df["airline"] == "US Airways","AIRLINE"] = "US Airways Inc."
tweets_df.loc[tweets_df["airline"] == "United", "AIRLINE"] = "United Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Virgin America", "AIRLINE"] = "Virgin America"

In [ ]:
# # Clean the tweet_created column to get the Date
# tweets_df['DATE'] = pd.to_datetime(tweets_df['tweet_created'])
# tweets_df['DATE']=pd.DatetimeIndex(tweets_df['DATE']).date
# tweets_df = tweets_df.drop(columns=['tweet_created','airline'])

In [ ]:
# Another Method:
# tweets_df['DATE'] = tweets_df['tweet_created'].str[0:10]
# tweets_df['DATE'] = pd.to_datetime(tweets_df['DATE'])
# tweets_df.head()
# tweets_df = tweets_df.drop(columns=['tweet_created'])
# tweets_df.dtypes

In [ ]:
# replace NaN with "No comments"
tweets_df['negativereason'] = tweets_df['negativereason'].fillna('No comments')
tweets_df

In [ ]:
tweets_df['AIRLINE'].value_counts()

In [ ]:
tweets_df['airline_sentiment'].value_counts()

In [ ]:
tweets_df.groupby(['AIRLINE','airline_sentiment']).count()
# Apparently, United AirLines has the most negative reviews
# Virgin America has the least nagative reviews

In [ ]:
# If we want to change NaN valule to 0
# tweets_df.loc[tweets_df['set_of_numbers'].isnull(),'value_is_NaN'] = 'Yes'
# tweets_df.loc[tweets_df['set_of_numbers'].notnull(),'value_is_NaN'] = 'No'

# count_nan = tweets_df.loc[tweets_df['value_is_NaN']=='Yes'].count()
# print (count_nan)

## Connect to MongoDB

In [ ]:
import pymysql
pymysql.install_as_MySQLdb()
import pymongo
import MySQLdb

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.flights_db

In [ ]:
#airport code data
for index,row in airports_df.iterrows():
    print(row)
    db.airports_df.insert_one(dict(row))

In [ ]:
#tweet data table
for index,row in tweets_df.iterrows():
    print(row)
    db.tweets_df.insert_one(dict(row))

In [ ]:
#airline and flight delay tables merged
for index,row in flights_air.iterrows():
    print(row)
    db.flight.insert_one(dict(row))
    break

In [ ]:
for item in db.flight.find():
    print(item)

In [ ]:
# db.flight.find()